#### 1. Import libraries

In [ ]:
import torch
from torch import linalg as la
from torch import digamma, log, sqrt
from torch.special import gammaln, ndtr
torch.set_default_dtype(torch.float64)

import math
import numpy
from numpy import linalg as LA
import pandas
from copy import deepcopy
from scipy import special
from tqdm import tqdm
from scipy.stats import qmc
from scipy.special import gammaincinv
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, average_precision_score
from sklearn.metrics import root_mean_squared_error, mean_squared_error, r2_score

import import_ipynb
import data_analysis

#### 2. Binary linear classifiers

##### Minimization objective

In [ ]:
def minimize_binary_clf_gn(initial_parameters, X, y, n_points, learning_rate, Lambda, max_iters, weights, print_flag):
    # data shape
    n_samples, n_features = X.shape
    # get initial_parameters
    C, Epsilon, K_p, TAU_p, K, TAU, W, MU, SIGMA, w_ext, mu_ext, sigma_ext = initial_parameters

    optimizer = torch.optim.NAdam([{'params': [C, W, MU, w_ext, mu_ext]},
                                   {'params': [Epsilon, SIGMA, sigma_ext], 'lr': 0.05 * learning_rate},
                                   {'params': [K, TAU], 'lr': 20 * learning_rate}
                                  ], lr=learning_rate)

    terminated_conv, old_risk_bound = 0, 1000
    norm_X_square = la.norm(X, dim=1)**2
    Max_, max_ = 1000 * torch.ones(n_points), torch.ones(n_points)

    dim, n_row = 1, 60
    sampler = qmc.Halton(dim, scramble=True, seed=0)
    x_halton = sampler.random(n_row)

    for i in range(max_iters):
        optimizer.zero_grad(set_to_none=True)

        noncentral_parameter = (la.norm((C.reshape(n_points, 1, n_features) - X), axis=2) / Epsilon.reshape(n_points,1))**2
        inv_deno = sqrt(4.5 * n_features + (4.5 * noncentral_parameter**2 / (n_features + 2 * noncentral_parameter)))
        
        # Mat_Loss_in, Loss_ext
        Mat_Loss_in = 1 - ndtr( (y.reshape(n_samples, 1) * (torch.matmul(X, W.T) + MU)) / sqrt(SIGMA**2 + norm_X_square.reshape(n_samples, 1)) ).T
        Loss_ext = 1 - ndtr( (y * (torch.matmul(X, w_ext) + mu_ext)) / sqrt(sigma_ext**2 + norm_X_square) )
        
        # compute Mat_Omega, Mat_Omega_log
        BETA = gammaincinv(K, x_halton) / TAU
        Mat_Omega = ndtr( (((n_features + noncentral_parameter)**(-1/3) * inv_deno) * torch.ones(n_row, 1, 1)) * ((BETA / Epsilon).reshape(n_row, n_points, 1))**(2/3) - ((inv_deno - inv_deno**(-1)) * torch.ones(n_row, 1, 1)) )
        Mat_Omega, Mat_Omega_log = torch.mean(Mat_Omega, axis=0), torch.mean((log(BETA.reshape(n_row, n_points, 1)) * Mat_Omega), axis=0)
        
        # compute KL divergence
        kl = - n_features * math.log(torch.prod(Epsilon)) - math.log(torch.prod(SIGMA) * sigma_ext) + 0.5 * (n_features * torch.sum(Epsilon**2) + (torch.sum(SIGMA**2) + sigma_ext**2) - ((n_features + 1) * n_points + 1))
        kl = kl + 0.5 * ( (la.norm(W, axis=1)**2).sum() + (MU**2).sum() + la.norm(w_ext)**2 + mu_ext**2 + (la.norm(C, axis=1)**2).sum() )
        kl = kl + ( (K - K_p) * digamma(K) + gammaln(K_p) - gammaln(K) + K_p * log(TAU / TAU_p) + K * ((TAU_p - TAU) / TAU) ).sum()
        
        # compute risk_bound
        loss = torch.sum(Mat_Omega * Mat_Loss_in, axis=0) + torch.prod(1 - Mat_Omega, axis=0) * Loss_ext
        risk_bound = torch.mean(loss * weights) + (1 / Lambda) * kl
    
        # compute gradient over all parameters but K
        risk_bound.backward()
        
        # compute gradient over K parameter
        Mat_Omega_complement_modified = torch.where((1 - Mat_Omega) == 0, 1e-4, (1 - Mat_Omega))
        Mat_Omega_prod = torch.prod((torch.zeros(n_points, 1, n_samples) + Mat_Omega_complement_modified), axis=1) / Mat_Omega_complement_modified
        grad_K = ((log(TAU) - digamma(K)).reshape(n_points,1) * Mat_Omega + Mat_Omega_log) * (Mat_Loss_in - Mat_Omega_prod * (torch.ones(n_points,1) * Loss_ext))
        K.grad = torch.mean(grad_K * (torch.ones(n_points,1) * weights.reshape(n_samples)), axis=1) + (1 / Lambda) * ((K - K_p) * torch.polygamma(1, K) + ((TAU_p - TAU) / TAU))
        
        optimizer.step()               # update gradient

        with torch.no_grad():
            K[:] = torch.clamp(K, min=K_p, max=Max_)                                  # contraint : K_p < K < Max_
            TAU[:] = torch.clamp(TAU, min=TAU_p, max=Max_)                            # contraint : TAU_p < TAU < Max_
            
            Epsilon[:] = torch.clamp(torch.abs(Epsilon), max=max_)                    # contraint : 0 < RHO_0 < max_
            SIGMA[:] = torch.clamp(torch.abs(SIGMA), max=max_)                        # contraint on 0 < SIGMA < 1
            sigma_ext[:] = torch.clamp(torch.abs(sigma_ext), max=torch.tensor([1.0])) # contraint on 0 < sigma_ext < 1

            # gradients and risk bound stop criterion
            if (i+1)%40 == 0 :
                if ((risk_bound / old_risk_bound) >= 0.95) or ((K / TAU**2).sum() > n_points) :
                    terminated_conv = 1
                    break;
                else:
                    old_risk_bound = risk_bound
                        
    if print_flag:
        if terminated_conv != 0:
            print(f'Converged in {i+1} iteration...')
        else:
            print(f'Terminated in {i+1} iteration... because maximum number of iterations has been exceeded') 
    return C.detach().numpy(), Epsilon.detach().numpy(), K.detach().numpy(), TAU.detach().numpy(), W.detach().numpy(), MU.detach().numpy(), SIGMA.detach().numpy(), w_ext.detach().numpy(), mu_ext.detach().numpy()[0], sigma_ext.detach().numpy()[0]


##### Definition of mixture models class

In [ ]:
class Pac_bayes_clf:
    def __init__(self, learning_rate=0.01, lambda_reg=100, C_ini=None, Epsilon_ini=None, W_ini=None, w_ext_ini=None, K_ini=None, TAU_ini=None, K_p=None, TAU_p=None, max_iters=2000, class_weights=None, print_flag=False):
        self.lr = learning_rate                                    # the step size at each iteration
        self.Lambda = lambda_reg                                   # the regularization rate (lambda_reg should be higher than 1)
        self.C_ini, self.Epsilon_ini = C_ini, Epsilon_ini          # initial mean vector C of the points of interest and its standard error Epsilon_ini
        self.W_ini, self.w_ext_ini = W_ini, w_ext_ini              # initial mean vector weights Wi and w_ext of the hyperplanes Vi and v_ext
        self.K_ini, self.TAU_ini = K_ini, TAU_ini                  # initial values of the shape and rate of locality parameter beta to find (beta = k/tau)
        self.K_p, self.TAU_p = K_p, TAU_p                          # prior value of the shape and rate (gamma distribution parameter)
        self.max_iters = max_iters                                 # maximum number of iterations before stopping independently of any early stopping
        self.weights = class_weights                               # to take into account the skewed distribution of the classes (if necessary), it just a flag
        self.print_flag = print_flag                               # flag to print some infos
        self.C, self.Epsilon = None, None                          # point of interest and it's standard deviation
        self.W, self.MU, self.SIGMA = None, None, None             # model parameters to be find in n localities
        self.w_ext, self.mu_ext, self.sigma_ext = None, None, None # model external parameters to be find out of n localities
        self.K, self.TAU, self.BETA = None, None, None             # the locality parameters beta to find beta = (k, tau)
        self.weights_vet = None                                    # to take into account the weights the target (if necessary)

    def fit(self, X, y):
        # data shape and number of vicinity points
        n_samples, n_features = X.shape
        n_points = len(self.K_p)

        # compute class_weight
        if self.weights is None:
            self.weights_vet = numpy.ones(n_samples) 
        else:
            class_weights = compute_class_weight(class_weight='balanced', classes=numpy.unique(y), y=y)
            self.weights_vet = numpy.where(y == numpy.unique(y)[0], class_weights[0], class_weights[1])

        # initialize parameters
        if self.C_ini is None:
            self.C_ini, self.Epsilon_ini = torch.randn(n_points, n_features), torch.rand(n_points)
            self.W_ini, self.w_ext_ini = torch.randn(n_points, n_features), torch.randn(n_features)
            self.K_ini, self.TAU_ini = 50 * torch.rand(n_points), 50 * torch.rand(n_points)

        self.C, self.Epsilon = deepcopy(self.C_ini), deepcopy(self.Epsilon_ini)
        self.W, self.MU, self.SIGMA = deepcopy(self.W_ini), torch.zeros(n_points), torch.ones(n_points)
        self.w_ext, self.mu_ext, self.sigma_ext = deepcopy(self.w_ext_ini), torch.tensor([0.0]), torch.tensor([1.0])
        self.K, self.TAU = deepcopy(self.K_ini), deepcopy(self.TAU_ini)

        # concatenate (initialize parameters)
        starting_point = (self.C.requires_grad_(), self.Epsilon.requires_grad_(), torch.from_numpy(self.K_p), torch.from_numpy(self.TAU_p), self.K, self.TAU.requires_grad_(), self.W.requires_grad_(), self.MU.requires_grad_(), self.SIGMA.requires_grad_(), self.w_ext.requires_grad_(), self.mu_ext.requires_grad_(), self.sigma_ext.requires_grad_())
        # convert data to tensor
        X_, y_, self.weights_vet = torch.from_numpy(X), torch.from_numpy(y), torch.from_numpy(self.weights_vet)
        # minimized params
        self.C, self.Epsilon, self.K, self.TAU, self.W, self.MU, self.SIGMA, self.w_ext, self.mu_ext, self.sigma_ext = minimize_binary_clf_gn(starting_point, X_, y_, n_points, self.lr, self.Lambda, self.max_iters, self.weights_vet, self.print_flag)
        self.BETA = numpy.round(self.K / self.TAU, 6)
            
    def predict(self, X): 
        n_points = self.C.shape[0]
        Mat_dists = LA.norm((self.C.reshape(n_points, 1, self.C.shape[1]) - X), axis=2)

        W_params = numpy.concatenate((self.W, self.w_ext.reshape(1, self.w_ext.size)), axis=0)
        MU_params = numpy.concatenate((self.MU, numpy.array([self.mu_ext])), axis=0)
        approx = (numpy.matmul(X, W_params.T) + MU_params).T
        
        # get idx_W for each samples
        idx_W = numpy.argmin((Mat_dists + numpy.where(Mat_dists <= self.BETA.reshape(n_points,1), 0, 1)), axis=0)
        number_of_overlap_region = numpy.sum(numpy.where(Mat_dists <= self.BETA.reshape(n_points,1), 1, 0), axis=0)
        idx_W[numpy.where(number_of_overlap_region == 0)[0]] = n_points

        mask = numpy.nonzero( (numpy.ones((n_points+1,1)) * idx_W) == (numpy.arange(n_points+1)).reshape(n_points+1,1) )
        mask = (mask[0][numpy.argsort(mask[1])], numpy.sort(mask[1]))
        return numpy.where(numpy.sign(approx[mask]) <= 0, -1, 1)
    
    def score(self, X, y):
        return numpy.round(accuracy_score(y, self.predict(X))*100, 4)

    def risk_bound(self, X, y):
        # get data shape
        n_samples, n_features = X.shape
        n_points = self.K.size
        
        dim, n_row = 1, 60
        sampler = qmc.Halton(dim, scramble=True, seed=0)
        x_halton = sampler.random(n_row)
        
        # compute Mat_Omega
        noncentral_parameter = (LA.norm((self.C.reshape(n_points, 1, n_features) - X), axis=2) / self.Epsilon.reshape(n_points,1))**2
        inv_deno = numpy.sqrt(4.5 * n_features + (4.5 * noncentral_parameter**2 / (n_features + 2 * noncentral_parameter)))
        
        BETA = gammaincinv(self.K, x_halton) / self.TAU
        Mat_Omega = special.ndtr( (((n_features + noncentral_parameter)**(-1/3) * inv_deno) * numpy.ones((n_row, 1, 1))) * ((BETA / self.Epsilon).reshape((n_row, n_points, 1)))**(2/3) - ((inv_deno - inv_deno**(-1)) * numpy.ones((n_row, 1, 1))) )
        Mat_Omega = numpy.mean(Mat_Omega, axis=0)

        # Mat_loss_in and Loss_ext
        norm_X_square = LA.norm(X, axis=1)**2
        Mat_loss_in = 1 - special.ndtr( (y.reshape(n_samples, 1) * (numpy.dot(X, self.W.T) + self.MU)) / numpy.sqrt(self.SIGMA**2 + norm_X_square.reshape(n_samples, 1)) ).T
        Loss_ext = 1 - special.ndtr( (y * (numpy.dot(X, self.w_ext) + self.mu_ext)) / numpy.sqrt(self.sigma_ext**2 + norm_X_square) )
        
        # compute empirical risk
        ER = numpy.mean(numpy.sum(Mat_Omega * Mat_loss_in, axis=0) + numpy.prod(1 - Mat_Omega, axis=0) * Loss_ext)
        
        # compute KL divergence
        gaussian_kl_var = - n_features * math.log(numpy.prod(self.Epsilon)) - math.log(numpy.prod(self.SIGMA) * self.sigma_ext) + 0.5 * (n_features * numpy.sum(self.Epsilon**2) + (numpy.sum(self.SIGMA**2) + self.sigma_ext**2) - ((n_features + 1) * n_points + 1))
        gaussian_kl = 0.5 * (numpy.sum(LA.norm(self.W, axis=1)**2, axis=0) + numpy.sum(self.MU**2, axis=0) + LA.norm(self.w_ext)**2 + self.mu_ext**2 + numpy.sum(LA.norm(self.C, axis=1)**2, axis=0))
        gamma_kl = numpy.sum( (self.K - self.K_p) * special.psi(self.K) + special.gammaln(self.K_p) - special.gammaln(self.K) + self.K_p * numpy.log(self.TAU / self.TAU_p) + self.K * ((self.TAU_p - self.TAU) / self.TAU) )
        return ER, ER + (1 / self.Lambda) * (gaussian_kl_var + gaussian_kl + gamma_kl)
        

##### Choice of Lambda

In [ ]:
def build_mixture_clf(X_train, X_val, y_train, y_val, T=10, lr=0.01, lambda_param=1000, K_p=None, TAU_p=None, max_iters=2000, class_weights=None, print_flag=False): 
    old_val_ER, old_val_risk = 1e10, 1e10
    for i in range(T):
        mixture_clf = Pac_bayes_clf(learning_rate=lr, lambda_reg=lambda_param, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, class_weights=class_weights, print_flag=print_flag)
        mixture_clf.fit(X_train, y_train)
        new_val_ER, new_val_risk = mixture_clf.risk_bound(X_val, y_val)
        
        if (new_val_ER < old_val_ER) and (new_val_risk < old_val_risk):
            Initial_parameters = (mixture_clf.C_ini, mixture_clf.Epsilon_ini, mixture_clf.W_ini, mixture_clf.w_ext_ini, mixture_clf.K_ini, mixture_clf.TAU_ini)
            val_score, old_val_ER, old_val_risk = mixture_clf.score(X_val, y_val), new_val_ER, new_val_risk
    return val_score, Initial_parameters

def lambda_validation_clf(X_train, X_val, y_train, y_val, lr=0.01, K_p=None, TAU_p=None, max_iters=2000, class_weights=None): 
    T, old_val_score = 10 * len(K_p), 0
    lambda_params = numpy.arange(1, 5.25, 0.25) * (1 / len(K_p)) * X_train.shape[0]

    i = 0
    pbar = tqdm(desc="Tuning Lambda ("+str(T)+" random restarts for each lambda) : ", total=len(lambda_params), position=0)
        
    while(i < len(lambda_params)) :
        lambda_param = lambda_params[i]
        torch.manual_seed(lambda_param)
        new_val_score, new_Initial_parameters = build_mixture_clf(X_train, X_val, y_train, y_val, T=T, lr=lr, lambda_param=lambda_param, K_p=K_p, TAU_p=TAU_p, 
                                                                  max_iters=max_iters, class_weights=class_weights, print_flag=False)
        if (new_val_score >= old_val_score) :
            lambda_param_, Initial_parameters, old_val_score = lambda_param, new_Initial_parameters, new_val_score
        
        pbar.update(1)
        i = i + 1

        elapsed = pbar.format_dict["elapsed"]
        if elapsed > 1800:
            break;
            
    pbar.close()
    return lambda_param_, Initial_parameters
    

##### Mixture Program

In [ ]:
def Mixture_clf(data, target_name, n_points=2, train_size=0.75, lr=0.01, lambda_param=100, max_iters=2000, scale=False, lambda_validation=False, times=1, return_flag='simple'):
    # uncouping X and y
    X, y = data_analysis.uncouping_x_y(data.copy(), target_name)
    # get imbalanced flag
    #imbalanced = data_analysis.get_imbalanced_flag(y)
    #class_weights = 'balanced' if imbalanced == True else None
    class_weights = None
    print('***************** Mixtures of transparent local models with unknown points of interest *****************')
    print(f'Training_set = {round((train_size * 100))}%, Validation_set = {round(((1 - train_size)/2) * 100)}%, Test_set = {round(((1 - train_size)/2) * 100)}%, n_points = {n_points}, class_weights = {class_weights}, scale = {scale}, lambda_validation = {lambda_validation}, times = {times}')
    
    for i in tqdm(numpy.arange(times), desc="For Random Data Split = "+str(times)+" …", total=times, position=0):
        # split the dataset X into the training set X_train and temporary set X_temp
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size = train_size, stratify=y, random_state=i)
        # split the dataset X_temp into the validation set X_val and testing set X_test
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size = 0.5, stratify=y_temp, random_state=0)
        X_train, X_val, X_test = data_analysis.reset_index_data(data_1=X_train, data_2=X_val, data_3=X_test, data_4=None)

        # data encoding (target encoding for category variables) and scaling (example : 'TargetEncoder', 'OrdinalEncoder', etc...)
        if scale == True:
            X_train_enc, X_val_enc, X_test_enc = data_analysis.data_processing(xtrain=X_train.copy(), ytrain=y_train.copy(), xtest_1=X_val.copy(), xtest_2=X_test.copy(), xtest_3=None, enc_type='OneHotEncoder', scale_type='Standardscaler')
            X_train_enc, X_val_enc, X_test_enc = X_train_enc.values.copy(), X_val_enc.values.copy(), X_test_enc.values.copy()
        else:
            X_train_enc, X_val_enc, X_test_enc = X_train.values.copy(), X_val.values.copy(), X_test.values.copy()
    
        # define gamma_priors
        K_p, TAU_p = 2.0 * numpy.ones(n_points), (1 / 10) * numpy.ones(n_points)
        
        # finding the best lambda by cross validation on data
        if lambda_validation == True:
            lambda_param, Initial_parameters = lambda_validation_clf(X_train_enc.copy(), X_val_enc.copy(), y_train.copy(), y_val.copy(), lr=lr, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, class_weights=class_weights)
        else :
            torch.manual_seed(lambda_param)
            Acc_score, Initial_parameters = build_mixture_clf(X_train_enc.copy(), X_val_enc.copy(), y_train.copy(), y_val.copy(), T=10 * n_points, lr=lr, lambda_param=lambda_param, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, class_weights=class_weights, print_flag=False)
        
        # fitting model
        mixture_clf = Pac_bayes_clf(learning_rate=lr, lambda_reg=lambda_param, C_ini=Initial_parameters[0], Epsilon_ini=Initial_parameters[1], W_ini=Initial_parameters[2], w_ext_ini=Initial_parameters[3], K_ini=Initial_parameters[4], TAU_ini=Initial_parameters[5], 
                                    K_p=K_p.copy(), TAU_p=TAU_p.copy(), max_iters=max_iters, class_weights=class_weights, print_flag=False)
        mixture_clf.fit(X_train_enc, y_train)
 
        # prediction
        y_train_preds = mixture_clf.predict(X_train_enc)
        y_val_preds = mixture_clf.predict(X_val_enc)
        y_test_preds = mixture_clf.predict(X_test_enc)
        
        # compute risk for each dataset
        risk_bound_set = []
        risk_bound_set.append(mixture_clf.risk_bound(X_train_enc, y_train))
        risk_bound_set.append(mixture_clf.risk_bound(X_val_enc, y_val))
        risk_bound_set.append(mixture_clf.risk_bound(X_test_enc, y_test))
        risk_bound_set = numpy.round(numpy.array(risk_bound_set), 4)
    
        # get summary
        summary_random = results_summary_clf(y_train, y_val, y_test, y_train_preds, y_val_preds, y_test_preds, risk_bound_set[:,0], risk_bound_set[:,1])

        if (i == 0):
            C_argsort = mixture_clf.C[:, 0].argsort()
            C_random_state, Epsilon_random_state = mixture_clf.C[C_argsort], mixture_clf.Epsilon[C_argsort]
            W_random_state, MU_random_state, SIGMA_random_state = mixture_clf.W[C_argsort], mixture_clf.MU[C_argsort], mixture_clf.SIGMA[C_argsort]
            w_ext_random_state, mu_ext_random_state, sigma_ext_random_state = mixture_clf.w_ext, mixture_clf.mu_ext, mixture_clf.sigma_ext
            K_random_state, TAU_random_state, BETA_random_state = mixture_clf.K[C_argsort], mixture_clf.TAU[C_argsort], mixture_clf.BETA[C_argsort]
            summary_random_state = summary_random
        else:
            C_argsort = mixture_clf.C[:, 0].argsort()
            C_random_state += mixture_clf.C[C_argsort]
            Epsilon_random_state += mixture_clf.Epsilon[C_argsort]
            W_random_state += mixture_clf.W[C_argsort]
            MU_random_state += mixture_clf.MU[C_argsort]
            SIGMA_random_state += mixture_clf.SIGMA[C_argsort]
            w_ext_random_state += mixture_clf.w_ext
            mu_ext_random_state += mixture_clf.mu_ext
            sigma_ext_random_state += mixture_clf.sigma_ext
            K_random_state += mixture_clf.K[C_argsort]
            TAU_random_state += mixture_clf.TAU[C_argsort]
            BETA_random_state += mixture_clf.BETA[C_argsort]
            summary_random_state += summary_random

        print(f'C = {mixture_clf.C[C_argsort]}, Epsilon = {numpy.round(mixture_clf.Epsilon[C_argsort], 6)}, lambda_param = {lambda_param}')
        print(f'K = {mixture_clf.K[C_argsort]}, TAU = {numpy.round(mixture_clf.TAU[C_argsort], 6)}')
        print(f'W = {mixture_clf.W[C_argsort]}, MU = {numpy.round(mixture_clf.MU[C_argsort], 6)}')
        print(f'w_ext = {mixture_clf.w_ext}, mu_ext = {round(mixture_clf.mu_ext, 6)}')
        print(f'SIGMA = {numpy.round(mixture_clf.SIGMA[C_argsort], 6)}, sigma_ext = {round(mixture_clf.sigma_ext, 6)}')

    C, Epsilon = (C_random_state / times), (Epsilon_random_state / times)
    W, MU, SIGMA = (W_random_state / times), (MU_random_state / times), (SIGMA_random_state / times)
    w_ext, mu_ext, sigma_ext = (w_ext_random_state / times), (mu_ext_random_state / times), (sigma_ext_random_state / times)
    K, TAU, BETA = (K_random_state / times), (TAU_random_state / times), (BETA_random_state / times)
    summary = (summary_random_state / times).astype('float64')
    summary = summary.round(4)    

    print(f'*********** END ***********')
    print(f'C = {C}, Epsilon = {numpy.round(Epsilon, 6)}, K = {K}, TAU = {numpy.round(TAU, 6)}')
    print(f'W = {W}, MU = {numpy.round(MU, 6)}')
    print(f'w_ext = {w_ext}, mu_ext = {round(mu_ext, 6)}')
    print(f'SIGMA = {numpy.round(SIGMA, 6)}, sigma_ext = {round(sigma_ext, 6)}')
    
    if return_flag=='simple':
        return lambda_param, C, Epsilon, W, w_ext, MU, mu_ext, K, TAU, BETA, summary
    else :
        return lambda_param, C, Epsilon, W, w_ext, MU, mu_ext, K, TAU, BETA, summary, X_train_enc, X_test_enc, y_train.copy(), y_test.copy(), y_train_preds.copy(), y_test_preds.copy()

##### Performance measurement

In [ ]:
def results_summary_clf(ytrain_true, yval_true, ytest_true, ytrain_pred, yval_pred, ytest_pred, Gibbs_risk_set, risk_bound_set):
    # for global model
    Summary_index = ['Training set', 'Validation set', 'Testing set']
    Summary_columns = ['Accuracy', 'F1_score', 'Precision_score', 'Gibbs_risk', 'Risk_bound']
    Summary_results = pandas.DataFrame(index=Summary_index, columns=Summary_columns)

    # performance of the global model
    Summary_results.loc[Summary_index[0], Summary_columns[0]] = round(accuracy_score(ytrain_true, ytrain_pred)*100, 2)
    Summary_results.loc[Summary_index[0], Summary_columns[1]] = round(f1_score(ytrain_true, ytrain_pred)*100, 2)
    Summary_results.loc[Summary_index[0], Summary_columns[2]] = round(average_precision_score(ytrain_true, ytrain_pred)*100, 2)
    Summary_results.loc[Summary_index[0], Summary_columns[3]] = Gibbs_risk_set[0]
    Summary_results.loc[Summary_index[0], Summary_columns[4]] = risk_bound_set[0]
    
    Summary_results.loc[Summary_index[1], Summary_columns[0]] = round(accuracy_score(yval_true, yval_pred)*100, 2)
    Summary_results.loc[Summary_index[1], Summary_columns[1]] = round(f1_score(yval_true, yval_pred)*100, 2)
    Summary_results.loc[Summary_index[1], Summary_columns[2]] = round(average_precision_score(yval_true, yval_pred)*100, 2)
    Summary_results.loc[Summary_index[1], Summary_columns[3]] = Gibbs_risk_set[1]
    Summary_results.loc[Summary_index[1], Summary_columns[4]] = risk_bound_set[1]
    
    Summary_results.loc[Summary_index[2], Summary_columns[0]] = round(accuracy_score(ytest_true, ytest_pred)*100, 2)
    Summary_results.loc[Summary_index[2], Summary_columns[1]] = round(f1_score(ytest_true, ytest_pred)*100, 2)
    Summary_results.loc[Summary_index[2], Summary_columns[2]] = round(average_precision_score(ytest_true, ytest_pred)*100, 2)
    Summary_results.loc[Summary_index[2], Summary_columns[3]] = Gibbs_risk_set[2]
    Summary_results.loc[Summary_index[2], Summary_columns[4]] = risk_bound_set[2]
    
    return Summary_results

#### 3. Linear regression

##### Minimization objective

In [ ]:
def minimize_reg_gn(initial_parameters, X, y, n_points, learning_rate, Lambda, max_iters, weights, print_flag):
    # data shape
    n_samples, n_features = X.shape
    # get initial_parameters
    C, Epsilon, K_p, TAU_p, K, TAU, W, RHO, MU, SIGMA, w_ext, rho_ext, mu_ext, sigma_ext = initial_parameters

    optimizer = torch.optim.NAdam([{'params': [C, W, MU, w_ext, mu_ext]},
                                   {'params': [Epsilon, RHO, SIGMA, rho_ext, sigma_ext], 'lr': 0.05 * learning_rate},
                                   {'params': [K, TAU], 'lr': 20 * learning_rate}
                                  ], lr=learning_rate)

    terminated_conv, old_risk_bound = 0, 1000
    norm_X_square = la.norm(X, dim=1)**2
    Max_, max_ = 1000 * torch.ones(n_points), torch.ones(n_points)

    dim, n_row = 1, 60
    sampler = qmc.Halton(dim, scramble=True, seed=0)
    x_halton = sampler.random(n_row)
    
    for i in range(max_iters):
        optimizer.zero_grad()
        
        # compute Mat_Omega
        noncentral_parameter = (la.norm((C.reshape(n_points, 1, n_features) - X), axis=2) / Epsilon.reshape(n_points,1))**2
        inv_deno = sqrt(4.5 * n_features + (4.5 * noncentral_parameter**2 / (n_features + 2 * noncentral_parameter)))

        # Mat_Loss_in, Loss_ext
        Mat_Loss_in = (norm_X_square.reshape(n_samples, 1) * RHO**2 + SIGMA**2 + ((torch.matmul(X, W.T) + MU) - y.reshape(n_samples, 1))**2).T
        Loss_ext = norm_X_square * rho_ext**2 + sigma_ext**2 + (torch.matmul(X, w_ext) + mu_ext - y)**2

        # compute Mat_Omega, Mat_Omega_log
        BETA = gammaincinv(K, x_halton) / TAU
        Mat_Omega = ndtr( (((n_features + noncentral_parameter)**(-1/3) * inv_deno) * torch.ones(n_row, 1, 1)) * ((BETA / Epsilon).reshape(n_row, n_points, 1))**(2/3) - ((inv_deno - inv_deno**(-1)) * torch.ones(n_row, 1, 1)) )
        Mat_Omega, Mat_Omega_log = torch.mean(Mat_Omega, axis=0), torch.mean((log(BETA.reshape(n_row, n_points, 1)) * Mat_Omega), axis=0)
        
        # compute KL divergence
        kl = - n_features * math.log(torch.prod(RHO) * rho_ext * torch.prod(Epsilon)) - math.log(torch.prod(SIGMA) * sigma_ext) + 0.5 * (n_features * (torch.sum(RHO**2) + rho_ext**2 + torch.sum(Epsilon**2)) + (torch.sum(SIGMA**2) + sigma_ext**2) - ((2*n_features + 1) * n_points + (n_features + 1)))
        kl = kl + 0.5 * ( (la.norm(W, axis=1)**2).sum() + (MU**2).sum() + la.norm(w_ext)**2 + mu_ext**2 + (la.norm(C, axis=1)**2).sum() )
        kl = kl + ( (K - K_p) * digamma(K) + gammaln(K_p) - gammaln(K) + K_p * log(TAU / TAU_p) + K * ((TAU_p - TAU) / TAU) ).sum()

        # compute risk_bound
        loss = torch.sum(Mat_Omega * Mat_Loss_in, axis=0) + torch.prod(1 - Mat_Omega, axis=0) * Loss_ext
        risk_bound = torch.mean(loss * weights) + (1 / Lambda) * kl
    
        # compute gradient over all parameters but K
        risk_bound.backward()
        
        # compute gradient over K parameter
        Mat_Omega_complement_modified = torch.where((1 - Mat_Omega) == 0, 1e-4, (1 - Mat_Omega))
        Mat_Omega_prod = torch.prod((torch.zeros(n_points, 1, n_samples) + Mat_Omega_complement_modified), axis=1) / Mat_Omega_complement_modified
        grad_K = ((log(TAU) - digamma(K)).reshape(n_points,1) * Mat_Omega + Mat_Omega_log) * (Mat_Loss_in - Mat_Omega_prod * (torch.ones(n_points,1) * Loss_ext))
        K.grad = torch.mean(grad_K * (torch.ones(n_points,1) * weights.reshape(n_samples)), axis=1) + (1 / Lambda) * ((K - K_p) * torch.polygamma(1, K) + ((TAU_p - TAU) / TAU))

        optimizer.step()                        # update gradient
        
        with torch.no_grad():
            K[:] = torch.clamp(K, min=K_p, max=Max_)                                  # contraint on K_p < K < Max_
            TAU[:] = torch.clamp(TAU, min=TAU_p, max=Max_)                            # contraint on TAU_p < TAU < Max_

            Epsilon[:] = torch.clamp(torch.abs(Epsilon), max=max_)                       # contraint on RHO_0 > 0
            RHO[:] = torch.clamp(torch.abs(RHO), max=max_)                            # contraint on RHO>0
            SIGMA[:] = torch.clamp(torch.abs(SIGMA), max=max_)                        # contraint on SIGMA>0
            rho_ext[:] = torch.clamp(torch.abs(rho_ext), max=torch.tensor([1.0]))     # contraint on rho_ext>0
            sigma_ext[:] = torch.clamp(torch.abs(sigma_ext), max=torch.tensor([1.0])) # contraint on sigma_ext>0

            # gradients and risk bound stop criterion
            if (i+1)%40 == 0 :
                if ((risk_bound / old_risk_bound) >= 0.95) or ((K / TAU**2).sum() > n_points) :
                    terminated_conv = 1
                    break;
                else:
                    old_risk_bound = risk_bound
                    
    if print_flag:
        if terminated_conv != 0:
            print(f'Converged in {i+1} iteration...')
        else:
            print(f'Terminated in {i+1} iteration... because maximum number of iterations has been exceeded') 
    return C.detach().numpy(), Epsilon.detach().numpy(), K.detach().numpy(), TAU.detach().numpy(), W.detach().numpy(), RHO.detach().numpy(), MU.detach().numpy(), SIGMA.detach().numpy(), w_ext.detach().numpy(), rho_ext.detach().numpy()[0], mu_ext.detach().numpy()[0], sigma_ext.detach().numpy()[0]
    

##### Definition of mixture models class

In [ ]:
class Pac_bayes_reg:
    def __init__(self, learning_rate=0.01, lambda_reg=0.1, C_ini=None, Epsilon_ini=None, K_ini=None, TAU_ini=None, K_p=None, TAU_p=None, max_iters=1000, weights=None, print_flag=False):
        self.lr = learning_rate                                    # the step size at each iteration
        self.Lambda = lambda_reg                                   # the regularization rate (lambda_reg should be higher than 1)
        self.C_ini, self.Epsilon_ini = C_ini, Epsilon_ini          # initial mean vector C of the points of interest and its standard error Epsilon_ini
        self.K_ini, self.TAU_ini = K_ini, TAU_ini                  # initial values of the shape and rate of locality parameter beta to find (beta = k/tau)
        self.K_p, self.TAU_p = K_p, TAU_p                          # prior value of the shape and rate (gamma distribution parameter)
        self.max_iters = max_iters                                 # maximum number of iterations before stopping independently of any early stopping
        self.weights_vet = weights                                 # to take into account the skewed distribution of the classes (if necessary)
        self.print_flag = print_flag                               # flag to print some infos
        self.C, self.Epsilon = None, None                          # point of interest and it's standard deviation
        self.W, self.RHO, self.MU, self.SIGMA = None, None, None, None                 # model parameters to be find in n localities
        self.w_ext, self.rho_ext, self.mu_ext, self.sigma_ext = None, None, None, None # model external parameters to be find out of n localities
        self.K, self.TAU, self.BETA = None, None, None                                 # the locality parameters beta to find beta = (k, tau)
        
    def fit(self, X, y):
        # data shape and number of vicinity points
        n_samples, n_features = X.shape
        n_points = self.K_p.size

        # compute weight
        self.weights_vet = numpy.ones(n_samples) if self.weights_vet is None else self.weights_vet

        # initialize parameters
        if self.C_ini is None:
            self.C_ini, self.Epsilon_ini = torch.randn(n_points, n_features), torch.rand(n_points)
            self.K_ini, self.TAU_ini = 50 * torch.rand(n_points), 50 * torch.rand(n_points)

        self.C, self.Epsilon = deepcopy(self.C_ini), deepcopy(self.Epsilon_ini)
        self.K, self.TAU = deepcopy(self.K_ini), deepcopy(self.TAU_ini)
        
        self.W, self.RHO = torch.zeros(n_points, n_features), torch.ones(n_points)
        self.MU, self.SIGMA = torch.zeros(n_points), torch.ones(n_points)
        self.w_ext, self.rho_ext = torch.zeros(n_features), torch.tensor([1.0])
        self.mu_ext, self.sigma_ext = torch.tensor([0.0]), torch.tensor([1.0])

        # concatenate (initialize parameters)
        starting_point = (self.C.requires_grad_(), self.Epsilon.requires_grad_(), torch.from_numpy(self.K_p), torch.from_numpy(self.TAU_p), self.K, self.TAU.requires_grad_(), self.W.requires_grad_(), self.RHO.requires_grad_(), self.MU.requires_grad_(), self.SIGMA.requires_grad_(), self.w_ext.requires_grad_(), self.rho_ext.requires_grad_(), self.mu_ext.requires_grad_(), self.sigma_ext.requires_grad_())
        # convert data to tensor
        X_, y_, self.weights_vet = torch.from_numpy(X), torch.from_numpy(y), torch.from_numpy(self.weights_vet)
        # minimized params
        self.C, self.Epsilon, self.K, self.TAU, self.W, self.RHO, self.MU, self.SIGMA, self.w_ext, self.rho_ext, self.mu_ext, self.sigma_ext = minimize_reg_gn(starting_point, X_, y_, n_points, self.lr, self.Lambda, self.max_iters, self.weights_vet, self.print_flag)
        self.BETA = numpy.round(self.K / self.TAU, 6)
            
    def predict(self, X):
        n_points = self.C.shape[0]
        Mat_dists = LA.norm((self.C.reshape(n_points, 1, self.C.shape[1]) - X), axis=2)

        W_params = numpy.concatenate((self.W, self.w_ext.reshape(1, self.w_ext.size)), axis=0)
        MU_params = numpy.concatenate((self.MU, numpy.array([self.mu_ext])), axis=0)
        approx = (numpy.matmul(X, W_params.T) + MU_params).T
        
        # get idx_W for each samples
        idx_W = numpy.argmin((Mat_dists + numpy.where(Mat_dists <= self.BETA.reshape(n_points,1), 0, 1)), axis=0)
        number_of_overlap_region = numpy.sum(numpy.where(Mat_dists <= self.BETA.reshape(n_points,1), 1, 0), axis=0)
        idx_W[numpy.where(number_of_overlap_region == 0)[0]] = n_points

        mask = numpy.nonzero( (numpy.ones((n_points+1,1)) * idx_W) == (numpy.arange(n_points+1)).reshape(n_points+1,1) )
        mask = (mask[0][numpy.argsort(mask[1])], numpy.sort(mask[1]))
        return approx[mask]
    
    def score(self, X, y):
        return numpy.round(r2_score(y_true=y, y_pred=self.predict(X)), 6)

    def risk_bound(self, X, y):
        # get data shape
        n_samples, n_features = X.shape
        n_points = self.K.size
        
        dim, n_row = 1, 60
        sampler = qmc.Halton(dim, scramble=True, seed=0)
        x_halton = sampler.random(n_row)

        # compute Mat_Omega
        noncentral_parameter = (LA.norm((self.C.reshape(n_points, 1, n_features) - X), axis=2) / self.Epsilon.reshape(n_points,1))**2
        inv_deno = numpy.sqrt(4.5 * n_features + (4.5 * noncentral_parameter**2 / (n_features + 2 * noncentral_parameter)))
        
        BETA = gammaincinv(self.K, x_halton) / self.TAU
        Mat_Omega = special.ndtr( (((n_features + noncentral_parameter)**(-1/3) * inv_deno) * numpy.ones((n_row, 1, 1))) * ((BETA / self.Epsilon).reshape((n_row, n_points, 1)))**(2/3) - ((inv_deno - inv_deno**(-1)) * numpy.ones((n_row, 1, 1))) )
        Mat_Omega = numpy.mean(Mat_Omega, axis=0)
        
        # Mat_loss_in and Loss_out
        norm_X_square = LA.norm(X, axis=1)**2
        Mat_loss_in = (norm_X_square.reshape(n_samples, 1) * self.RHO**2 + self.SIGMA**2 + ((numpy.dot(X, self.W.T) + self.MU) - y.reshape(n_samples, 1))**2).T
        Loss_out = norm_X_square * self.rho_ext**2 + self.sigma_ext**2 + (numpy.dot(X, self.w_ext) + self.mu_ext - y)**2

        # compute empirical risk
        ER = numpy.mean(numpy.sum(Mat_Omega * Mat_loss_in, axis=0) + numpy.prod(1 - Mat_Omega, axis=0) * Loss_out)
        
        # compute KL divergence
        gaussian_kl = - n_features * math.log(numpy.prod(self.RHO) * self.rho_ext * numpy.prod(self.Epsilon)) - math.log(numpy.prod(self.SIGMA) * self.sigma_ext) + 0.5 * (n_features * (numpy.sum(self.RHO**2) + self.rho_ext**2 + numpy.sum(self.Epsilon**2)) + (numpy.sum(self.SIGMA**2) + self.sigma_ext**2) - ((2*n_features + 1) * n_points + (n_features + 1)))
        gaussian_kl += 0.5 * (numpy.sum(LA.norm(self.W, axis=1)**2, axis=0) + numpy.sum(self.MU**2, axis=0) + LA.norm(self.w_ext)**2 + self.mu_ext**2 + numpy.sum(LA.norm(self.C, axis=1)**2, axis=0))
        gamma_kl = numpy.sum( (self.K - self.K_p) * special.psi(self.K) + special.gammaln(self.K_p) - special.gammaln(self.K) + self.K_p * numpy.log(self.TAU / self.TAU_p) + self.K * ((self.TAU_p - self.TAU) / self.TAU) )
        return ER, ER + (1 / self.Lambda) * (gaussian_kl + gamma_kl)
        

##### Choice of Lambda

In [ ]:
def build_mixture_reg(X_train, X_val, y_train, y_val, T=10, lr=0.01, lambda_param=1000, K_p=None, TAU_p=None, max_iters=1000, weights=None, print_flag=False):
    old_val_ER, old_val_risk = 1e10, 1e10
    for i in range(T):
        mixture_reg = Pac_bayes_reg(learning_rate=lr, lambda_reg=lambda_param, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, weights=weights, print_flag=print_flag)
        mixture_reg.fit(X_train, y_train)
        new_val_ER, new_val_risk = mixture_reg.risk_bound(X_val, y_val)

        if (new_val_ER < old_val_ER) and (new_val_risk < old_val_risk):
            Initial_parameters = (mixture_reg.C_ini, mixture_reg.Epsilon_ini, mixture_reg.K_ini, mixture_reg.TAU_ini)
            val_score, old_val_ER, old_val_risk = mixture_reg.score(X_val, y_val), new_val_ER, new_val_risk
    return val_score, Initial_parameters

def lambda_validation_reg(X_train, X_val, y_train, y_val, lr=0.01, K_p=None, TAU_p=None, max_iters=1000, weights=None): 
    T, old_val_score = 10 * len(K_p), -1e10
    lambda_params = numpy.arange(1, 5.25, 0.25) * (1 / len(K_p)) * X_train.shape[0]

    i = 0
    pbar = tqdm(desc="Tuning Lambda ("+str(T)+" random restarts for each lambda) : ", total=len(lambda_params), position=0)
        
    while(i < len(lambda_params)) :
        lambda_param = lambda_params[i]
        torch.manual_seed(lambda_param)
        new_val_score, new_Initial_parameters = build_mixture_reg(X_train, X_val, y_train, y_val, T=T, lr=lr, lambda_param=lambda_param, K_p=K_p, TAU_p=TAU_p, 
                                                                  max_iters=max_iters, weights=weights, print_flag=False)
        if (new_val_score >= old_val_score):
            lambda_param_, Initial_parameters, old_val_score = lambda_param, new_Initial_parameters, new_val_score
        
        pbar.update(1)
        i = i + 1

        elapsed = pbar.format_dict["elapsed"]
        if elapsed > 1800:
            break;
            
    pbar.close()
    return lambda_param_, Initial_parameters


##### Mixture Program

In [ ]:
def Mixture_reg(data, target_name, synthetic_data_flag=False, n_points=2, train_size=0.75, lr=0.01, lambda_param=1e-3, max_iters=1000, scale=False, lambda_validation=False, times=1, check_multicollinearity=True, return_flag='simple'):
    # uncouping X and y reg
    X, y = data_analysis.uncouping_x_y_reg(data, target_name, synthetic_data_flag=synthetic_data_flag)
    # get imbalanced flag
    weights = None
    print('***************** Mixtures of transparent local models without given points of interest *****************')
    print(f'Training_set = {round((train_size * 100))}%, Validation_set = {round(((1 - train_size)/2) * 100)}%, Test_set = {round(((1 - train_size)/2) * 100)}%, n_points = {n_points}, weights = {weights}, scale = {scale}, lambda_validation = {lambda_validation}, times = {times}, check_multicollinearity = {check_multicollinearity}')

    for i in tqdm(numpy.arange(times), desc="For Random Data Split = "+str(times)+" …", total=times, position=0):
        # split the dataset X into the training set X_train and temporary set X_temp
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size = train_size, random_state=i)
        # split the dataset X_temp into the validation set X_val and testing set X_test
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size = 0.5, random_state=0)
        X_train, X_val, X_test = data_analysis.reset_index_data(data_1=X_train, data_2=X_val, data_3=X_test, data_4=None)    

        # data encoding (target encoding for category variables) and scaling (example : 'TargetEncoder', 'OrdinalEncoder', etc...)
        if scale == True:
            X_train_enc, X_val_enc, X_test_enc = data_analysis.data_processing(xtrain=X_train.copy(), ytrain=y_train.copy(), xtest_1=X_val.copy(), xtest_2=X_test.copy(), xtest_3=None, enc_type='OneHotEncoder', scale_type='Standardscaler', check_multicollinearity=check_multicollinearity)
            X_train_enc, X_val_enc, X_test_enc = X_train_enc.values.copy(), X_val_enc.values.copy(), X_test_enc.values.copy()
        else:
            X_train_enc, X_val_enc, X_test_enc = X_train.values.copy(), X_val.values.copy(), X_test.values.copy()

        # define gamma_priors
        K_p, TAU_p = 2.0 * numpy.ones(n_points), (1 / 10) * numpy.ones(n_points)
        
        # finding the best lambda by cross validation on data
        if lambda_validation == True:
            lambda_param, Initial_parameters = lambda_validation_reg(X_train_enc.copy(), X_val_enc.copy(), y_train.copy(), y_val.copy(), lr=lr, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, weights=weights)
        else :
            torch.manual_seed(lambda_param)
            R2_score, Initial_parameters = build_mixture_reg(X_train_enc.copy(), X_val_enc.copy(), y_train.copy(), y_val.copy(), T=10 * n_points, lr=lr, lambda_param=lambda_param, K_p=K_p, TAU_p=TAU_p, max_iters=max_iters, weights=weights, print_flag=False)
            
        # fitting model
        mixture_reg = Pac_bayes_reg(learning_rate=lr, lambda_reg=lambda_param, C_ini=Initial_parameters[0], Epsilon_ini=Initial_parameters[1], K_ini=Initial_parameters[2], TAU_ini=Initial_parameters[3], K_p=K_p.copy(), TAU_p=TAU_p.copy(), max_iters=max_iters, weights=weights, print_flag=False)
        mixture_reg.fit(X_train_enc, y_train)
        
        # prediction
        y_train_preds = mixture_reg.predict(X_train_enc)
        y_val_preds = mixture_reg.predict(X_val_enc)
        y_test_preds = mixture_reg.predict(X_test_enc)
        
        # compute risk for each dataset
        risk_bound_set = []
        risk_bound_set.append(mixture_reg.risk_bound(X_train_enc, y_train))
        risk_bound_set.append(mixture_reg.risk_bound(X_val_enc, y_val))
        risk_bound_set.append(mixture_reg.risk_bound(X_test_enc, y_test))
        risk_bound_set = numpy.round(numpy.array(risk_bound_set), 4)

        # get summary
        summary_random = results_summary_reg(y_train, y_val, y_test, y_train_preds, y_val_preds, y_test_preds, risk_bound_set[:,0], risk_bound_set[:,1])

        if (i == 0):
            C_argsort = mixture_reg.C[:, 0].argsort()
            C_random_state, Epsilon_random_state = mixture_reg.C[C_argsort], mixture_reg.Epsilon[C_argsort]
            W_random_state, RHO_random_state, MU_random_state, SIGMA_random_state = mixture_reg.W[C_argsort], mixture_reg.RHO[C_argsort], mixture_reg.MU[C_argsort], mixture_reg.SIGMA[C_argsort]
            w_ext_random_state, rho_ext_random_state, mu_ext_random_state, sigma_ext_random_state = mixture_reg.w_ext, mixture_reg.rho_ext, mixture_reg.mu_ext, mixture_reg.sigma_ext
            K_random_state, TAU_random_state, BETA_random_state = mixture_reg.K[C_argsort], mixture_reg.TAU[C_argsort], mixture_reg.BETA[C_argsort]
            summary_random_state = summary_random
        else:
            C_argsort = mixture_reg.C[:, 0].argsort()
            C_random_state += mixture_reg.C[C_argsort]
            Epsilon_random_state += mixture_reg.Epsilon[C_argsort]
            W_random_state += mixture_reg.W[C_argsort]
            RHO_random_state += mixture_reg.RHO[C_argsort]
            MU_random_state += mixture_reg.MU[C_argsort]
            SIGMA_random_state += mixture_reg.SIGMA[C_argsort]
            w_ext_random_state += mixture_reg.w_ext
            rho_ext_random_state += mixture_reg.rho_ext
            mu_ext_random_state += mixture_reg.mu_ext
            sigma_ext_random_state += mixture_reg.sigma_ext
            K_random_state += mixture_reg.K[C_argsort]
            TAU_random_state += mixture_reg.TAU[C_argsort]
            BETA_random_state += mixture_reg.BETA[C_argsort]
            summary_random_state += summary_random

        print(f'C = {mixture_reg.C[C_argsort]}, Epsilon = {numpy.round(mixture_reg.Epsilon[C_argsort], 6)}, lambda_param = {lambda_param}')
        print(f'W = {mixture_reg.W[C_argsort]}, MU = {numpy.round(mixture_reg.MU[C_argsort], 6)}')
        print(f'w_ext = {mixture_reg.w_ext}, mu_ext = {round(mixture_reg.mu_ext, 6)}')
        print(f'RHO = {mixture_reg.RHO[C_argsort]}, SIGMA = {numpy.round(mixture_reg.SIGMA[C_argsort], 6)}')
        print(f'rho_ext = {mixture_reg.rho_ext}, sigma_ext = {round(mixture_reg.sigma_ext, 6)}')

    C, Epsilon = (C_random_state / times), (Epsilon_random_state / times)
    W, RHO, MU, SIGMA = (W_random_state / times), (RHO_random_state / times), (MU_random_state / times), (SIGMA_random_state / times)
    w_ext, rho_ext, mu_ext, sigma_ext = (w_ext_random_state / times), (rho_ext_random_state / times), (mu_ext_random_state / times), (sigma_ext_random_state / times)
    K, TAU, BETA = (K_random_state / times), (TAU_random_state / times), (BETA_random_state / times)
    summary = (summary_random_state / times).astype('float64')
    summary = summary.round(4)

    print(f'*********** END ***********')
    print(f'C = {C}, Epsilon = {numpy.round(Epsilon, 6)}')
    print(f'W = {W}, MU = {numpy.round(MU, 6)}')
    print(f'w_ext = {w_ext}, mu_ext = {round(mu_ext, 6)}')
    print(f'RHO = {RHO}, SIGMA = {numpy.round(SIGMA, 6)}')
    print(f'rho_ext = {rho_ext}, sigma_ext = {round(sigma_ext, 6)}')
    
    if return_flag=='simple':
        return lambda_param, C, Epsilon, W, w_ext, MU, mu_ext, K, TAU, BETA, summary
    else :
        return lambda_param, C, Epsilon, W, w_ext, MU, mu_ext, K, TAU, BETA, summary, X_train_enc, X_test_enc, y_train.copy(), y_test.copy(), y_train_preds.copy(), y_test_preds.copy()
        

##### Performance measurement

In [ ]:
def results_summary_reg(ytrain_true, yval_true, ytest_true, ytrain_pred, yval_pred, ytest_pred, Gibbs_risk_set, risk_bound_set):
    # for global model
    Summary_index = ['Training set', 'Validation set', 'Testing set']
    Summary_columns = ['R2_score', 'RMSE', 'MSE', 'Gibbs_risk', 'Risk_bound']
    Summary_results = pandas.DataFrame(index=Summary_index, columns=Summary_columns)

    # performance of the global model
    Summary_results.loc[Summary_index[0], Summary_columns[0]] = round(r2_score(ytrain_true, ytrain_pred), 4)
    Summary_results.loc[Summary_index[0], Summary_columns[1]] = round(root_mean_squared_error(ytrain_true, ytrain_pred), 4)
    Summary_results.loc[Summary_index[0], Summary_columns[2]] = round(mean_squared_error(ytrain_true, ytrain_pred), 4)
    Summary_results.loc[Summary_index[0], Summary_columns[3]] = Gibbs_risk_set[0]
    Summary_results.loc[Summary_index[0], Summary_columns[4]] = risk_bound_set[0]
    
    Summary_results.loc[Summary_index[1], Summary_columns[0]] = round(r2_score(yval_true, yval_pred), 4)
    Summary_results.loc[Summary_index[1], Summary_columns[1]] = round(root_mean_squared_error(yval_true, yval_pred), 4)
    Summary_results.loc[Summary_index[1], Summary_columns[2]] = round(mean_squared_error(yval_true, yval_pred), 4)
    Summary_results.loc[Summary_index[1], Summary_columns[3]] = Gibbs_risk_set[1]
    Summary_results.loc[Summary_index[1], Summary_columns[4]] = risk_bound_set[1]
    
    Summary_results.loc[Summary_index[2], Summary_columns[0]] = round(r2_score(ytest_true, ytest_pred), 4)
    Summary_results.loc[Summary_index[2], Summary_columns[1]] = round(root_mean_squared_error(ytest_true, ytest_pred), 4)
    Summary_results.loc[Summary_index[2], Summary_columns[2]] = round(mean_squared_error(ytest_true, ytest_pred), 4)
    Summary_results.loc[Summary_index[2], Summary_columns[3]] = Gibbs_risk_set[2]
    Summary_results.loc[Summary_index[2], Summary_columns[4]] = risk_bound_set[2]
    
    return Summary_results